In [ ]:
import pandas as pd
import numpy as np
import datatable as dt

nlinesfile = 3140000
nlinesrandomsample = 300000
lines2skip = np.random.choice(np.arange(1,nlinesfile+1), (nlinesfile-nlinesrandomsample), replace=False)

#train = pd.read_csv('../input/ubiquant-market-prediction/train.csv', skiprows=lines2skip)
train = dt.fread('../input/ubiquant-market-prediction/train.csv').to_pandas()
test = pd.read_csv('../input/ubiquant-market-prediction/example_test.csv')
sub = pd.read_csv('../input/ubiquant-market-prediction/example_sample_submission.csv')

display(train)
display(test)
display(sub)

In [ ]:
None_values= train.isna().sum()
None_values[None_values > 0]

In [ ]:
print('row_id unique len: ', len(train['row_id'].unique()))
print('time_id unique len: ', len(train['time_id'].unique()))
print('investment_id unique len: ', len(train['investment_id'].unique()))

In [ ]:
train['time_id'] = train['time_id'].astype('int')
train['investment_id'] = train['investment_id'].astype('int')

In [ ]:
NUM_FEATURES = list(train.loc[:,train.dtypes==np.int].columns)
NUM_FEATURES_2 = list(train.loc[:,train.dtypes==np.float].columns)
NUM_FEATURES.extend(NUM_FEATURES_2)

FEATURES = list(train.columns)
CAT_FEATURES = [feature for feature in FEATURES if feature not in NUM_FEATURES]

NUM_FEATURES.remove('target')
CAT_FEATURES.remove('row_id')
IGNORE_COLS = ['row_id']
print(CAT_FEATURES)
print(NUM_FEATURES)

# P-Values

**Find the important columns that has p-values higher than 0.05**

In [ ]:
from scipy.stats import pearsonr

p_values_list = []
for c in NUM_FEATURES:
    p = round(pearsonr(train.loc[:,'target'], train.loc[:,c])[1], 4)
    p_values_list.append(p)

p_values_df = pd.DataFrame(p_values_list, columns=['target'], index=NUM_FEATURES)
def p_value_warning_background(cell_value):
    highlight = 'background-color: lightcoral;'
    default = ''
    if cell_value > 0.05:
            return highlight
    return default

p_values_df_high = p_values_df[p_values_df['target'] > 0.03]
p_values_df_high.style.applymap(p_value_warning_background)

In [ ]:
IMPORTANT_COLS = list(p_values_df[p_values_df['target'] > 0.05].index)

In [ ]:
print(IMPORTANT_COLS)

# **Skewedness**

**Drop columns with values in high skew.** 

In [ ]:
from scipy.stats import skew 
 
drop_2 = [column for column in train.columns if skew(train.loc[:, column]).abs() > 1]

In [ ]:
IGNORE_COLS.extend(drop_2)

# Ignore Columns

In [ ]:
IGNORE_COLS.remove('time_id')
IGNORE_COLS.remove('investment_id')

In [ ]:
IGNORE_COLS.remove('target')

In [ ]:
print(IGNORE_COLS)

In [ ]:
len(list(set(IGNORE_COLS)))

# Important Cols

**Important columns shall not be dropped though they're in IGNORE_COLS list.**

In [ ]:
print(IMPORTANT_COLS)

In [ ]:
len(IMPORTANT_COLS)